# Question 1

- This problem is aimed at building a simple recommendation system "People You Might Know" with Spark

### Construct baskets

In [1]:
from pyspark import SparkConf, SparkContext
import re
from itertools import combinations

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile("q1/data/soc-LiveJournal1Adj.txt")

In [3]:
slines = lines.map(lambda line: \
                    (int(re.split(r'\t+', line)[0]),\
                    re.split(r'[^\w]+', re.split(r'\t+', line)[1:][0])))

In [4]:
def str2int(v): 
    '''string list to int list'''
    if v != ['']:
        return [int(i) for i in v]
    else:
        return v

baskets = slines.mapValues(str2int)

In [5]:
baskets_unkeyed = baskets.flatMap(lambda x: [x[1]])\
                         .filter(lambda x: x!=[''])\
                         .filter(lambda x: len(x) > 1)

### Count mutual friends

In [65]:
direct_friends = baskets.flatMapValues(lambda x: x)\
                        .map(lambda x: (x, 1))

In [67]:
count_mutual = baskets_unkeyed.flatMap(lambda x: \
                    [(key, 1) for key in combinations(x, 2)])\
                              .subtract(direct_friends)\
                              .reduceByKey(lambda n1,n2: n1+n2)\
                              .sortBy(lambda x: (-x[1], x[0][1]))

In [68]:
collects = count_mutual.collect()

In [69]:
recmd = {}
for mutual in collects:
    if mutual[0][0] not in recmd.keys():
        recmd[mutual[0][0]] = [mutual[0][1]]
    elif len(recmd[mutual[0][0]]) < 10:
        recmd[mutual[0][0]].append(mutual[0][1])
        
    if mutual[0][1] not in recmd.keys():
        recmd[mutual[0][1]] = [mutual[0][0]]
    elif len(recmd[mutual[0][1]]) < 10:
        recmd[mutual[0][1]].append(mutual[0][0])

In [70]:
original_keys = baskets.keys().collect()

In [71]:
for usr in original_keys:
    if usr not in recmd.keys():
        recmd[usr] = ['']

### Save results

In [72]:
dictlist = []
for key, value in recmd.items():
    temp = [key,value]
    dictlist.append(temp)
dictlist.sort()

In [73]:
with open('q1/q1_output.txt', 'w') as f:
    for line in dictlist:
        f.write(str(line[0]))
        f.write('\t')
        for rec in line[1]:
            f.write(str(rec))
            if rec != line[1][-1]: f.write(',')
        f.write('\n')

# Question 2

- This problem is aimed at 